In [1]:
!pip install markdownify duckduckgo-search ipywidgets "transformers[agents]" --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# import requests
# from transformers.agents.llm_engine import HfApiEngine, MessageRole, get_clean_message_list

# openai_role_conversions = {  # Keep this for message cleaning
#     MessageRole.TOOL_RESPONSE: "user",
# }

# class OllamaHfApiWrapper(HfApiEngine):
#     def __init__(self, model_name, temperature, base_url="http://localhost:11434/"):
#         super().__init__(model_name) 
#         self.base_url = base_url
#         self.model_name = model_name
#         self.temperature = temperature

#     def query(self, payload):
#         messages = payload.get("inputs", [])
#         messages = get_clean_message_list(messages, role_conversions=openai_role_conversions)
#         data = {
#             "model": self.model_name,
#             "messages": messages,
#             "stream": False, # Set to false for non-streaming responses
#             "options": {"temperature": self.temperature},
#         }

#         headers = {"Content-Type": "application/json"}

#         try:
#             response = requests.post(self.base_url + "/api/chat", json=data, headers=headers, timeout=120)

#             response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
#             response_json = response.json()
#             ## llm_output = response_json["choices"][0]["message"]["content"].strip()
#             llm_output = response_json["message"]["content"].strip()
#             return [{"generated_text": llm_output}] # HfApiEngine expects a list of dictionaries

#         except requests.exceptions.RequestException as e:
#             print(f"Error during Ollama request: {e}")
#             return [{"generated_text": ""}]  # Return empty string in case of error
#         except (KeyError, IndexError) as e:
#             print(f"Error parsing Ollama response: {e}. Raw response: {response.text}")
#             return [{"generated_text": ""}]

# # Example usage:
# ollama_engine = OllamaHfApiWrapper(model_name="phi3:medium", temperature=0, base_url="http://localhost:11434/")

model = "Qwen/Qwen2.5-72B-Instruct"

In [4]:
import re
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
from transformers.agents import tool


@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = md(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [5]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

Hugging Face - Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu
move to sidebar
hide

Navigation

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Contac


In [6]:
from transformers.agents import (
    ReactCodeAgent,
    ReactJsonAgent,
    HfApiEngine,
    ManagedAgent,
)
from transformers.agents.search import DuckDuckGoSearchTool

llm_engine = HfApiEngine(model)

web_agent = ReactJsonAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    llm_engine=llm_engine,
    max_iterations=10,
)

In [7]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="Runs web searches for you. Give it your query as an argument.",
)

In [8]:
manager_agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=["time", "datetime"],
)

In [9]:
manager_agent.run("Can you list me the related works about Gans?")

======== New task ========
Can you list me the related works about Gans?
=== Agent thoughts:
Thought: I will use the `search` tool to find works related to GANs (Generative Adversarial Networks).
>>> Agent is executing the code below:
related_works = search(request="related works about Generative Adversarial Networks")
print(related_works)
====
======== New task ========
You're a helpful agent named 'search'.
You have been submitted this task by your manager.
---
Task:
related works about Generative Adversarial Networks
---
You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much information as possible so that they have a clear understanding of the answer.

Your final_answer WILL HAVE to contain these parts:
### 1. Task outcome (short version):
### 2. Task outcome (extremely detailed version):
### 3. Additional context (if relevant):

Put all these in your final_answer tool, everything that you do not pass as an argument to final_

['1. **Generative Adversarial Networks (2014)** by Ian Goodfellow et al. - The original paper that introduced GANs. [Link](https://arxiv.org/abs/1406.2661)',
 '2. **A Review on Generative Adversarial Networks: Algorithms, Theory, and Applications (2021)** by Zhang et al. - A comprehensive review of GAN methods. [Link](https://ieeexplore.ieee.org/document/9625798)',
 '3. **Generative Adversarial Networks: An Overview (2017)** by Odena et al. - An overview of GANs and their theoretical foundations. [Link](https://arxiv.org/pdf/1710.07035.pdf)',
 '4. **Recent Progress on Generative Adversarial Networks (GANs): A Survey (2019)** by Mao et al. - A survey of recent progress and advancements in GANs. [Link](https://ieeexplore.ieee.org/document/8667290)']

In [16]:
manager_agent.run("Who is Eyüp Sercan UYGUR?")

======== New task ========
Who is Eyüp Sercan UYGUR?
=== Agent thoughts:
Thought: To find out who Eyüp Sercan UYGUR is, I will use the `search` tool to gather information about this person.
>>> Agent is executing the code below:
search_results = search("Eyüp Sercan UYGUR")
print(search_results)
====
======== New task ========
You're a helpful agent named 'search'.
You have been submitted this task by your manager.
---
Task:
Eyüp Sercan UYGUR
---
You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much information as possible so that they have a clear understanding of the answer.

Your final_answer WILL HAVE to contain these parts:
### 1. Task outcome (short version):
### 2. Task outcome (extremely detailed version):
### 3. Additional context (if relevant):

Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be lost.
And even if your task resolution is not successful, please 

{'Full Name': 'Eyüp Serkan Uygur',
 'Profession': 'Web Developer, AI Professional, Software Engineer',
 'Location': 'Istanbul, Turkey',
 'Education': 'Istanbul Technical University',
 'Professional Profile': 'Senior software engineer at Orion Innovation',
 'Professional Network': 'LinkedIn, GitHub, HuggingFace, HackerNoon',
 'Interests / Skills': 'Web development, AI, software engineering, JavaScript, Ruby, Python, CSS, HTML, animations, user experience, 3D vision, CSS animations, web design',
 'Publications and Models': 'Has published and shared work including VisionGPT-3D, a general multimodal agent for enhanced 3D vision understanding',
 'Personal Website': 'https://www.eyupsercanuygur.com/'}